# 数据安全攻击

## 成员推理攻击（Membership Inference Attack）
成员推理攻击的攻击目标是判断一个输入样本**是否存在于模型的训练数据集**中，可能会导致数据泄露、隐私泄露等风险。由于人工智能模型易过拟合训练数据、训练数据与测试数据分布的差异性等原因，使得深度学习模型易遭受成员推理攻击。

本节实验使用影子模型攻击来实现简单的成员推理攻击。其主要想法是，训练多个“影子模型”，通过影子模型来模拟受害者模型的表现，然后根据影子模型的训练情况构建攻击数据集。使用该攻击数据集训练的二分类模型可以判断一个输入样本是否是来自受害者模型的训练集。

其主要流程如下图所示：
<p align="center">
<img src="./src/overview.png" style="center" width="70%">
<center>影子模型攻击流程</center>
</p>

### 1. 训练受害者模型
首先使用 sklearn.datasets.load_wine() 加载葡萄酒分类数据集。

葡萄酒分类数据集是一个非常小且简单的数据集，其具体信息如下：
数据集包含 178 个样本，每个样本拥有 13 种不同的化学特征（即每个样本有 13 维特征），包括酸度、灰分，酒精浓度等。然后一共有三种不同类别的葡萄酒，对应标签0到2。

详细信息可参考[这个网页](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html#sklearn.datasets.load_wine)

In [ ]:
# 加载葡萄酒分类数据集
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm
import numpy as np
import random

data, target = load_wine(return_X_y=True)
# 将每一维度缩放至0到1
scaler = MinMaxScaler()
data = scaler.fit_transform(data)
# 查看数据集的形状
print("数据集的形状:", data.shape)
# 查看数据集的前5行
print("数据集的前5行:\n", data[:5])
# 查看数据集的标签
print("数据集的标签:", target[:5])

使用随机森林分类器来作为被攻击的目标模型（受害者模型）。
随机森林分类器的具体api参数内容可以参考[sklearn的官方网站](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

该分类器使用 .fit(X, y) 函数进行训练，使用 .predict(X) 函数来预测对应标签，使用 .predict_proba(X) 函数来获取每个标签的预测概率值。其中 X 的形状为（批次，特征数）。

In [ ]:
victim_model = RandomForestClassifier(n_estimators=100)
# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.4, random_state=42)
# 训练模型
victim_model.fit(X_train, y_train)
# 测试模型准确性
accuracy = victim_model.score(X_test, y_test)
print("模型准确性:", accuracy)

定义查询模型的函数：输入样本 $x \rightarrow$ 预测概率向量 $prob$ ，来模拟通过api访问模型的应用场景

In [ ]:
# 模拟通过api访问模型，得到模型的概率预测输出
def query_model(model, x):
    return model.predict_proba(x)

### 2. 合成数据集
由于攻击者无从得知受害者模型的训练集，而训练一个判断输入样本是否属于训练集的二分类模型需要相应的数据。因此，文章借助爬山算法来合成影子模型的训练数据集。具体算法如下：
<p align="center">
<img src="./src/synthesis.png" style="center" width="40%">
<center>合成数据集的算法</center>
</p>
值得注意的是，该算法运行时间较长，面对具有更多特征的数据集（如图像数据集）并不适用。文章提出在那些场景下，攻击者可能需要具有部分训练数据集的知识，如获得同分布但是无交集的数据集、或者可以得到扰动后的数据集。

In [ ]:
# 合成数据集，适用于无法获取真实数据集的情况
# 对于 features 较多的情况不适用
def synthesize_data(model, label, num_iter, conf_min, rej_max, k_max, k_min, num_features):
    """
    合成数据集算法
    
    参数:
    model: 目标模型
    label: 想要合成的样本标签
    num_iter: 最大迭代次数
    conf_min: 接受合成样本的最小置信度阈值
    rej_max: 连续拒绝的最大次数
    k_max, k_min: 每次迭代中随机改变的特征数量范围
    num_features: 特征总数
    
    返回:
    合成的样本特征向量或None
    """

    # TODO：根据伪代码实现合成数据集算法

    
    # 达到最大迭代次数仍未找到合适样本，返回None
    return None

合成影子模型训练所需数据集

In [ ]:
def get_synthetic_data(num_samples, num_features, model, num_classes, num_iter=1000, conf_min=0.8, rej_max=10, k_max=3, k_min=1):
    # 生成合成数据
    synthesize_data_list = []
    label_list = []
    for i in tqdm(range(num_samples // num_classes)):
        for label in range(num_classes):
            x = None
            while x is None:
                x = synthesize_data(model, label, num_iter
                                    , conf_min, rej_max, k_max, k_min, num_features)
            synthesize_data_list.append(x)
            label_list.append(label)
    return synthesize_data_list, label_list

num_classes = len(np.unique(y_train))

synthesize_data_train, synthesize_label_train = get_synthetic_data(num_samples=100, num_features=X_train.shape[1], model=victim_model, num_classes=num_classes)
# 将合成数据转换为 numpy 数组
synthesize_data_train, synthesize_label_train = np.array(synthesize_data_train), np.array(synthesize_label_train)

synthesize_data_test, synthesize_label_test = get_synthetic_data(num_samples=100, num_features=X_test.shape[1], model=victim_model, num_classes=num_classes)
synthesize_data_test, synthesize_label_test = np.array(synthesize_data_test), np.array(synthesize_label_test)

In [ ]:
# 保存合成数据集
import os
if not os.path.exists('./data'):
    os.makedirs('./data')
np.save('./data/synthesize_data_train.npy', synthesize_data_train)
np.save("./data/synthesis_label_train.npy", synthesize_label_train)
np.save('./data/synthesize_data_test.npy', synthesize_data_test)
np.save("./data/synthesis_label_test.npy", synthesize_label_test)
print("合成数据集已保存")

In [ ]:
synthesize_data_train = np.load('./data/synthesize_data_train.npy')
synthesize_label_train = np.load("./data/synthesis_label_train.npy")
synthesize_data_test = np.load('./data/synthesize_data_test.npy')
synthesize_label_test = np.load("./data/synthesis_label_test.npy")
print("合成数据集已加载")

In [ ]:
print(synthesize_data_train.shape)
print(synthesize_label_train.shape)

### 3. 训练影子模型
使用合成数据集训练影子模型，并且构建训练攻击模型所用的二分类数据集

由于攻击者不知道受害者模型训练的具体信息，而为了训练攻击的二分类模型（用于判断一个给定的输入是否在训练集中），攻击者需要构建一个（预测向量，是否在训练数据集中）的数据集来训练攻击模型。

攻击者可以通过构建多个影子模型，并且将合成数据集分为多份（彼此可重叠）来训练不同的影子模型，使得构建的攻击数据集更多样化。

而对于影子模型，攻击者知道一个样本是否是来自其训练数据集，因此可以构建训练攻击模型所需的数据集。影子模型的数量越多，产生的样本对越多，越利于攻击的二分类模型的训练。

对于攻击数据集，其每一个样本包含三部分，模型的预测向量 $\mathbf{y}$，真实标签 $y$，以及其是否在训练数据集中（0代表不在训练集中，1代表在训练集中）：
$$
D_{attack}=\{(\mathbf{y} ,y) , \text{in-or-out} \}_{i=1}^n
$$
其中 in-or-out 是0-1变量，代表该样本对是否在训练集中。

In [ ]:
# 将合成数据集分成 num_shared 份，可重叠
def split_dataset(dataset, true_label, num_shadows, num_classes, length):
    splits = []
    labels = []
    for i in range(num_shadows):
        split = []
        label = []
        for j in range(num_classes):
            idx = np.where(true_label == j)[0]
            if len(idx) < length:
                length = len(idx)
            selected_idx = np.random.choice(idx, size=length, replace=False)
            split.append(dataset[selected_idx])
            label.append(true_label[selected_idx])
        splits.append(np.concatenate(split, axis=0))
        labels.append(np.concatenate(label, axis=0))
    return splits, labels

In [ ]:
num_classes = len(np.unique(synthesize_label_train))
num_shadows = 20
attack_X = []
attack_y = []

shadow_train_list, train_label_list = split_dataset(synthesize_data_train, synthesize_label_train, num_shadows, num_classes, len(synthesize_data_train) // 2)
shadow_test_list, test_label_list = split_dataset(synthesize_data_test, synthesize_label_test, num_shadows, num_classes, len(synthesize_data_test) // 2)
# 训练影子模型
for i in range(num_shadows):
    shadow_model = RandomForestClassifier(n_estimators=100, random_state=i)
    shadow_model.fit(shadow_train_list[i], train_label_list[i])
    # 测试影子模型的准确性
    shadow_model_accuracy = shadow_model.score(shadow_test_list[i], test_label_list[i])
    print(f"Shadow model {i} accuracy: {shadow_model_accuracy}")

    # 使用影子模型生成攻击二分类数据集
    in_pred = query_model(shadow_model, shadow_train_list[i])
    in_features = np.hstack((in_pred, train_label_list[i].reshape(-1, 1)))
    attack_X.append(in_features)
    attack_y.append(np.ones((in_pred.shape[0], 1)))

    out_pred = query_model(shadow_model, shadow_test_list[i])
    out_features = np.hstack((out_pred, test_label_list[i].reshape(-1, 1)))
    attack_X.append(out_features)
    attack_y.append(np.zeros((out_pred.shape[0], 1)))
print("successfully generate attack dataset")

使用合成数据集构建二分类所用数据集

In [ ]:
attack_X = np.vstack(attack_X)
attack_y = np.vstack(attack_y).reshape(-1)
print("attack_X shape:", attack_X.shape)
print("attack_y shape:", attack_y.shape)

### 4. 训练攻击二分类模型
接下来，借助影子模型构建的攻击数据集，我们可以训练二分类模型。提出影子模型攻击的论文指出，该二分类模型是独立于之前的影子模型训练过程的，因此可以根据任务需求选择适合的模型进行训练。

这里我们依然采用随机森林分类器作为二分类的攻击模型。

In [ ]:
# 训练攻击模型
attack_model = RandomForestClassifier(n_estimators=100, random_state=42)
attack_model.fit(attack_X, attack_y)

接下来在真实的训练集以及测试集（即受害者模型所训练的数据集以及测试集）上测试我们的攻击效果。

In [ ]:
# 测试攻击模型
def test_attack_model(attack_model, victim_model, X_train, y_train, X_test, y_test):
    num_train = 0
    num_test = 0
    acc_train = 0
    acc_test = 0
    
    preds = query_model(victim_model, X_train)
    features = np.hstack((preds, y_train.reshape(-1, 1)))
    train_pred = attack_model.predict(features)
    num_train += len(train_pred)
    acc_train += np.sum(train_pred == 1)

    preds = query_model(victim_model, X_test)
    features = np.hstack((preds, y_test.reshape(-1, 1)))
    test_pred = attack_model.predict(features)
    num_test += len(test_pred)
    acc_test += np.sum(test_pred == 0)
    print("训练集攻击模型准确率:", acc_train / num_train)
    print("测试集攻击模型准确率:", acc_test / num_test)
    print("\n一些指标：")
    accuracy = (acc_train + acc_test) / (num_train + num_test)
    print("攻击模型准确率 accuracy:", accuracy)
    precision = acc_train / (acc_train + num_test - acc_test)
    print("精确率 precision:", precision)
    recall = acc_train / num_train
    print("召回率 recall:", recall)
    print("F1-score:", 2 * precision * recall / (precision + recall))
    
test_attack_model(attack_model, victim_model, X_train, y_train, X_test, y_test)

一个随机猜测的二分类判断器准确率为 $50%$，而我们的攻击分类器达到了 $60\%+$，说明该攻击有一定效果。

此外，影子模型攻击存在诸多限制：
- 攻击者需要知道目标模型结构
- 目标模型需要过拟合才能达到较好的攻击效果
- 局限于特征数少的分类任务

### 5. 简单的防御思路
同上一次实验中模型窃取的防御思路一致。可以采取模糊化模型输出向量的方式进行防御，使模型输出包含尽可能少的信息。可以将 query_model() 函数的输出换成模糊后的概率向量来模拟模糊化输出的过程。

In [ ]:
# 模拟通过api访问模型，得到模糊化后的概率预测输出
def query_model(model, x):
    result = model.predict_proba(x)
    # 模糊化处理，保留 1 位小数
    # 最大的预测向量不变，其余均设置为 0
    max_idx = np.argmax(result, axis=1)
    result = np.round(result, 1)
    # 将最大值以外的元素设置为 0
    for i in range(result.shape[0]):
        for j in range(result.shape[1]):
            if j != max_idx[i]:
                result[i, j] = 0.0
    return result

# 数据安全防御

## 差分隐私（DP）

差分隐私是一类重要的模型隐私保护技术，其通过**对样本注入噪声噪声**，使得差别只有一条记录的相邻数据集在模型推理时产生近似一致的输出概率，也就是“抹除”单个样本在模型中的区分度，从而保护模型隐私。

差分隐私对**单个样本隐私**的保护逻辑如下：假设数据集 $D$ 删除某样本 $y$ 后变为 $D'$ ，攻击者可能通过观察 $D$ 与 $D'$ 输入模型后输出结果的差异，推断样本 $y$ 的敏感信息（如收入、年龄等）。而差分隐私通过在**数据处理**的关键环节添加噪声（如数据收集、查询、发布阶段），使得攻击者无法通过模型输出的变化准确反推出特定样本的存在或属性--因为噪声的干扰会导致信息提取出现不可控偏差，从而将单个样本的隐私信息**模糊化**。

![差分隐私示意图](./imgs/dp1.jpg)

比如，对于 data-1 和 data-2 两次查询，未加差分隐私保护，则模型会输出 2 和 3 ，明显存在区别，则可以从这个区别分析特定样本属性，而加了差分隐私技术，模型可能会输出 2.5 左右的数据，使得 $D$ 和 $D'$ 相差的那个特定样本模糊化。

从数学角度出发，差分隐私确保了相邻数据集 $D$ 与 $D'$ 对任意输出结果 $O$ 的概率分布满足 $P[F(D) \in O] \le e^{\epsilon} \cdot P[F(D') \in O]$ （F是模型推理过程），也就是：$D$ 和 $D'$ 产生同一输出 $O$ 的**概率比值**被严格限制在 $[e^{-\epsilon}, e^{\epsilon}]$ 范围内， 通过严格的概率约束“抹除”单个样本对模型输出的决定性影响。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
import seaborn as sns
import math

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# 固定随机种子
torch.manual_seed(42)

# 设置随机种子以确保结果可复现
np.random.seed(42)

# 设置中文字体，确保图表中的中文能正常显示
plt.rcParams["font.family"] = ["SimHei", "Noto Sans CJK JP", "AR PL UMing JP"]
# plt.rcParams["font.family"] = ["Noto Sans CJK JP"]
plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题

# 确保在Jupyter Notebook中可以正常显示图表
%matplotlib inline

In [ ]:
def Differential_Privacy():
    # 可视化ε对隐私保护的影响
    epsilons = [0.1, 1, 5]
    x = np.linspace(0, 1, 100)
    
    plt.figure(figsize=(10, 6))
    for eps in epsilons:
        plt.plot(x, np.exp(eps) * x, label=f'ε = {eps}')
    
    plt.plot(x, x, 'k--', label='无隐私保护')
    plt.xlabel('原始概率 P[M(D\'∈S)]')
    plt.ylabel('扰动后概率上限 e^ε * P[M(D\'∈S)]')
    plt.title('不同ε值对差分隐私约束的影响')
    plt.legend()
    plt.grid(True)
    plt.show()
    

In [ ]:
Differential_Privacy()

图为不同 $\epsilon$ 值对原始概率 $P$ 的影响，$\epsilon$ 值越小，概率约束越严格，例如图中 $\epsilon=0.1$ 时，扰动后的概率最多是原始概率的 $e^{0.1} \approx 1.105$ 倍。

举个简单的例子：成员推理攻击（MIA）判断某个成员是否在数据集中，对于某个成员 A，在不添加扰动时很容易通过相邻数据集来进行判断。加入 $\epsilon=0.1$ 扰动，即使 A 在数据中，结果概率最多比没有 A 时高 $10.5\%$。

___

## 差分隐私实现
实现差分隐私，最重要的即是添加噪声，拉普拉斯机制（Laplace Mechanism）和高斯机制（Gaussian Mechanism）是两种在差分隐私中常见的**添加噪声**的方法。

下面我们对这两种方法进行逐一学习。

### 拉普拉斯机制

拉普拉斯机制通过向数据查询结果中**添加符合拉普拉斯分布($f(x|\mu, b) = \frac 1 {2b} e^{-\frac{|x - \mu|}{b}}$)的噪声**，来实现差分隐私保护。

拉普拉斯机制添加噪声小，且严格满足差分隐私定义，因此在简单的差分隐私保护场景中，通常使用拉普拉斯机制来实现差分隐私。

In [ ]:
def laplace_mechanism(true_value, sensitivity, epsilon):
    """
    实现Laplace机制
    
    参数:
    true_value (float): 真实查询值
    sensitivity (float): 查询的敏感度
    epsilon (float): 隐私预算
    
    返回:
    float: 添加噪声后的查询结果
    """

    # TODO：实现拉普拉斯机制

    # 计算Laplace分布的参数b = 敏感度/ε
    
    # 从Laplace分布中采样噪声
    
    # 添加噪声到真实值
    
    return noisy_value


def visualize_laplace_mechanism():
    # 真实值
    true_value = 100
    # 敏感度
    sensitivity = 1
    # 不同的隐私预算
    epsilons = [0.1, 1, 10]
    # 生成噪声样本
    num_samples = 1000
    samples = {}
    
    for eps in epsilons:
        samples[eps] = [laplace_mechanism(true_value, sensitivity, eps) for _ in range(num_samples)]
    
    plt.figure(figsize=(12, 6))
    
    for i, eps in enumerate(epsilons):
        plt.subplot(1, 3, i+1)
        sns.histplot(samples[eps], kde=True)
        plt.axvline(x=true_value, color='r', linestyle='--', label='真实值')
        plt.title(f'ε = {eps}')
        plt.xlabel('查询结果')
        plt.ylabel('频率')
        plt.legend()
    
    plt.tight_layout()
    plt.show()

我们来查看一下拉普拉斯机制的效果：

In [ ]:
visualize_laplace_mechanism()

实验结果符合差分隐私的定义：随着隐私预算 $\epsilon$ 的增大，噪声减小，查询结果会更接近真实值，但这样也会导致隐私保护程度降低；反之，隐私预算减小会增大隐私保护程度，但也会对模型的性能产生较大的影响。

### 高斯机制

与拉普拉斯机制类似，高斯机制通过向查询结果中添加高斯噪声来实现差分隐私。

对于函数 $f(D)$ （D为数据集），高斯机制 $M(D)$ 定义为：$M(D) = f(D) + \mathcal N(0, \delta ^2)$

In [ ]:
def gaussian_mechanism(true_value, sensitivity, epsilon, delta):
    """
    实现高斯机制
    
    参数:
    true_value (float): 真实查询值
    sensitivity (float): 查询的敏感度
    epsilon (float): 隐私预算
    delta (float): 失败概率
    
    返回:
    float: 添加噪声后的查询结果
    """

    # TODO：实现高斯机制

    # 计算高斯分布的标准差
    
    # 从高斯分布中采样噪声
    
    # 添加噪声到真实值
    
    return noisy_value

def visualize_gaussian_mechanism():
    # 真实值
    true_value = 100
    # 敏感度
    sensitivity = 1
    # 隐私参数
    epsilon = 1
    deltas = [1e-3, 1e-5, 1e-7]
    # 生成噪声样本
    num_samples = 1000
    samples = {}
    
    for delta in deltas:
        samples[delta] = [gaussian_mechanism(true_value, sensitivity, epsilon, delta) for _ in range(num_samples)]
    
    plt.figure(figsize=(12, 6))
    
    for i, delta in enumerate(deltas):
        plt.subplot(1, 3, i+1)
        sns.histplot(samples[delta], kde=True)
        plt.axvline(x=true_value, color='r', linestyle='--', label='真实值')
        plt.title(f'δ = {delta}')
        plt.xlabel('查询结果')
        plt.ylabel('频率')
        plt.legend()
    
    plt.tight_layout()
    plt.show()

我们来查看一下高斯机制的效果：

In [ ]:
visualize_gaussian_mechanism()

高斯机制通过 $\delta$ 参数实现了松弛差分隐私 $(\epsilon, \delta)-DP$ ， $\delta$ 给差分隐私提供了一个小概率隐私泄露的可能性，这样的好处是提高了数据的可用性。

高斯机制对高维数据或需要精确梯度的场景更加友好，因此更适合深度学习等场景。

## 差分隐私应用--人口数据分析

这一小节中，我们针对现实场景：**人口数据分析场景**，来展示差分隐私的实际应用。

In [ ]:
def demographic_data_example():
    """人口数据分析中的应用 + 差分隐私保护"""
    
    # 生成模拟人口数据
    n = 1000  # 样本量
    ages = np.random.normal(35, 10, n).astype(int)
    incomes = np.random.lognormal(10, 0.5, n).astype(int)
    education_levels = np.random.choice(['高中', '本科', '硕士', '博士'], size=n, 
                                     p=[0.4, 0.4, 0.15, 0.05])
    
    data = pd.DataFrame({
        '年龄': ages,
        '收入': incomes,
        '教育程度': education_levels
    })
    
    # 计算真实统计量
    true_mean_age = data['年龄'].mean()
    true_income_sum = data['收入'].sum()
    true_education_counts = data['教育程度'].value_counts()
    
    # 敏感度计算
    age_sensitivity = 1  # 均值查询的敏感度为1（添加或删除一个人的影响）
    income_sensitivity = data['收入'].max()  # 求和查询的敏感度为最大值
    education_sensitivity = 1  # 计数查询的敏感度为1
    
    # 设置隐私预算
    epsilon_total = 1.0
    
    # 分配隐私预算（简单平均分配）
    epsilon_age = epsilon_total / 3
    epsilon_income = epsilon_total / 3
    epsilon_education = epsilon_total / 3
    
    # 应用Laplace机制添加噪声
    noisy_mean_age = laplace_mechanism(true_mean_age, age_sensitivity, epsilon_age)
    noisy_income_sum = laplace_mechanism(true_income_sum, income_sensitivity, epsilon_income)
    
    # 对分类计数应用差分隐私
    noisy_education_counts = {}
    for level, count in true_education_counts.items():
        noisy_count = laplace_mechanism(count, education_sensitivity, epsilon_education)
        noisy_education_counts[level] = max(0, noisy_count)  # 确保计数非负
    
    # 结果可视化
    plt.figure(figsize=(15, 5))
    
    # 年龄均值
    plt.subplot(1, 3, 1)
    plt.bar(['真实值', '差分隐私值'], [true_mean_age, noisy_mean_age])
    plt.title('平均年龄')
    plt.ylabel('年龄')
    
    # 收入总和
    plt.subplot(1, 3, 2)
    plt.bar(['真实值', '差分隐私值'], [true_income_sum, noisy_income_sum])
    plt.title('总收入')
    plt.ylabel('收入')
    
    # 教育程度分布
    plt.subplot(1, 3, 3)
    levels = list(true_education_counts.index)
    true_counts = [true_education_counts[level] for level in levels]
    noisy_counts = [noisy_education_counts[level] for level in levels]
    
    x = np.arange(len(levels))
    width = 0.35
    
    plt.bar(x - width/2, true_counts, width, label='真实值')
    plt.bar(x + width/2, noisy_counts, width, label='差分隐私值')
    plt.xticks(x, levels)
    plt.title('教育程度分布')
    plt.ylabel('人数')
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    # 打印结果
    print(f"真实平均年龄: {true_mean_age:.2f}, 差分隐私平均年龄: {noisy_mean_age:.2f}")
    print(f"真实总收入: {true_income_sum:,}, 差分隐私总收入: {int(noisy_income_sum):,}")
    print("\n教育程度分布:")
    for level in levels:
        print(f"  {level}: 真实值={true_education_counts[level]}, 差分隐私值={int(noisy_education_counts[level])}")



In [ ]:
demographic_data_example()

使用差分隐私后，数据整体特征在保持不变的前提下，对数据进行了一定程度的扰动，在数据保持分析价值的情况下，对于单个样本来说，不是那么容易从相邻数据集中推断出来了，实现了对单个样本的隐私保护。

## 深度学习中的差分隐私

我们可以将差分隐私理论应用到深度学习中，保护深度学习模型训练和应用过程中的用户隐私，并尽可能维护模型性能。

在深度学习中，差分隐私的实现方式不是对数据样本直接添加噪声，也不是对模型输出添加噪声，而是在**模型训练过程**中，对模型的**梯度空间**进行**添加噪声**，**限制单个样本对梯度产生的影响**，防止攻击者通过梯度的变化推断出单个样本的属性。

在深度学习中，差分隐私训练逻辑为： 原始数据 $\xrightarrow{前向传播}$ 模型输出 $\xrightarrow{损失函数}$ 损失值 $\xrightarrow{反向传播}$ 原始梯度 $\xrightarrow[添加噪声]{梯度裁剪}$ 隐私合规梯度 $\xrightarrow{优化器}$ 模型参数更新

我们使用MNIST数据集来实现深度学习中的差分隐私：

In [ ]:
def load_mnist_data(batch_size=64, val_ratio=0.2):
    """加载MNIST数据集并分割训练/验证集"""
    # 数据预处理（标准化）
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))  # MNIST均值/标准差
    ])
    
    # 加载训练集
    train_dataset = datasets.MNIST(
        root='./data', train=True, download=True, transform=transform
    )
    val_size = int(len(train_dataset) * val_ratio)
    train_size = len(train_dataset) - val_size
    train_dataset, val_dataset = torch.utils.data.random_split(
        train_dataset, [train_size, val_size]
    )
    
    # 测试集
    test_dataset = datasets.MNIST(
        root='./data', train=False, download=True, transform=transform
    )
    
    # 创建数据加载器
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, val_loader, test_loader

def plot_mnist_examples(dataset, num_examples=6):
    fig, axes = plt.subplots(1, num_examples, figsize=(15, 2))
    for i in range(num_examples):
        img, label = dataset[i]
        axes[i].imshow(img.squeeze(), cmap='gray')
        axes[i].set_title(f'Label: {label}')
        axes[i].axis('off')
    plt.show()

In [ ]:
# 加载MNIST数据
train_loader, val_loader, test_loader = load_mnist_data(batch_size=128)

# 查看示例图像
plot_mnist_examples(datasets.MNIST('./data', train=True, download=True, transform=transforms.ToTensor()))

由于在差分隐私深度学习中，模型梯度的敏感性 $Sensitivity = max_{D, D'} ||\nabla L(D) - \nabla L(D')||$ 定义了相邻数据集的梯度的最大可能差异。

我们需要限制这个差异，否则梯度可能会因为个别样本而剧烈波动，导致需要添加**大量噪声**才能满足差分隐私，但是这样又会因为**过多噪声**导致模型性能严重下降。

一个有效的方法是在梯度空间进行加噪前，先设置梯度范数的上限（如 $L_2$ 范数不超过 1.0 ，把所有样本的梯度敏感性限制在固定范围内：

$$
clipped\_{grad} = grad \cdot min(1, \frac{clip\_norm}{||grad||})
$$

这样，无论某单个样本梯度有多大，其对整体梯度的贡献都会被限制在一个合理区间。

In [ ]:
# 梯度 L2 范数裁剪
def clip_gradient(grads, max_norm=1.0):
    """对梯度进行L2范数裁剪"""

    # TODO：补全梯度裁剪函数

    clipped = []
    for g in grads:
        if g is not None:
            # 计算梯度的L2范数
            # 如果范数大于 max_norm，则进行裁剪
            # 裁剪公式: clipped_g = g * (max_norm / norm)
        else:
            # 否则不进行裁剪
    return clipped


# 深度学习下的加噪函数

# 高斯噪声
def add_gaussian_noise(tensor, epsilon=1.0, delta=1e-5, sensitivity=1.0, **kargs):
    if delta <= 0:
        raise ValueError("Delta must be greater than 0")
    sigma = (math.sqrt(2 * math.log(1.25 / delta)) * sensitivity) / epsilon
    grad_std = max(torch.std(tensor).item(), 1e-6)
    sigma_adj = sigma * min(grad_std, 0.1)
    return tensor + torch.normal(0, sigma_adj, size=tensor.shape, device=tensor.device)

# 拉普拉斯噪声
def add_laplace_noise(tensor, epsilon=1.0, sensitivity=1.0, **kargs):
    scale = sensitivity / epsilon
    grad_std = max(torch.std(tensor).item(), 1e-6)
    scale_adj = scale * min(grad_std, 0.1)
    noise = np.random.laplace(0, scale_adj, size=tensor.shape)
    return tensor + torch.tensor(noise, device=tensor.device, dtype=tensor.dtype)

然后我们定义一个CNN模型用于差分隐私训练：

In [ ]:
def build_cnn_model():
    model = nn.Sequential(
        nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1), nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1), nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Flatten(),
        nn.Linear(32 * 7 * 7, 128), nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(128, 10)
    )
    return model

深度学习中，差分隐私主要是对梯度空间进行加噪，使得观察到的梯度和任何单个样本的真实梯度 $\nabla _\theta L$ 不可区分。

在模型训练函数中，我们根据差分隐私的定义，在模型**原始梯度**上进行裁剪和加噪，使得其符合隐私合规的梯度，再使用优化器来更新模型参数。

在训练阶段，我们平均为每一个epoch的训练分配合适的隐私预算。

In [ ]:
def run_training_epoch(model, data_loader, loss_fn, optimizer, dp_config, max_grad=1.0):
    """执行一个epoch的差分隐私训练"""
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0
    
    for X_batch, y_batch in data_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = loss_fn(outputs, y_batch)
        loss.backward()

        # TODO: 完成差分隐私训练部分

        # 1. 对反向传播求出的梯度进行裁剪

        # 2. 对梯度空间添加噪声（可自行选择 laplace 或 gaussian 噪声）

        # 3. 使用优化器对模型参数进行更新
        
        # 统计指标
        total_loss += loss.item()
        _, pred = torch.max(outputs, 1)
        total += y_batch.size(0)
        correct += (pred == y_batch).sum().item()
    
    return total_loss / len(data_loader), (correct / total) * 100

# 完整训练
def train_dp_model(train_loader, val_loader, dp_config, lr=0.01, max_grad=1.0, epochs=10):
    """差分隐私训练函数"""
    model = build_cnn_model()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.CrossEntropyLoss()
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': [], 'privacy': []}
    
    print(f"开始训练 | ε={dp_config['epsilon']}, 机制={dp_config['mechanism']}")
    for epoch in range(1, epochs+1):
        # 训练阶段
        train_loss, train_acc = run_training_epoch(
            model, train_loader, loss_fn, optimizer, dp_config, max_grad
        )
        
        # 验证阶段
        val_loss, val_acc = evaluate_model(model, val_loader, loss_fn)
        
        # 记录隐私预算
        privacy_spent = epoch * (dp_config['epsilon'] / epochs)  # 平均分配预算
        
        # 打印进度
        print(f"Epoch {epoch:2d}/{epochs:2d} | "
              f"Train: Loss {train_loss:.4f} Acc {train_acc:.2f}% | "
              f"Val:   Loss {val_loss:.4f} Acc {val_acc:.2f}% | "
              f"隐私消耗: {privacy_spent:.4f}")
        
        # 保存历史记录
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        history['privacy'].append(privacy_spent)
    
    return model, history

# 评估函数
def evaluate_model(model, data_loader, loss_fn):
    """评估模型在验证集上的性能"""
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for X_batch, y_batch in data_loader:
            outputs = model(X_batch)
            loss = loss_fn(outputs, y_batch)
            total_loss += loss.item()
            _, pred = torch.max(outputs, 1)
            total += y_batch.size(0)
            correct += (pred == y_batch).sum().item()
    
    return total_loss / len(data_loader), (correct / total) * 100

In [ ]:
# 绘制训练曲线
def plot_training_history(history: dict):
    """绘制训练/验证损失和准确率曲线"""
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # 损失曲线
    axes[0].plot(history['train_loss'], label='训练损失', c='blue')
    axes[0].plot(history['val_loss'], label='验证损失', c='orange', linestyle='--')
    axes[0].set_xlabel('Epoch'), axes[0].set_ylabel('Loss'), axes[0].grid(True), axes[0].legend()
    
    # 准确率曲线
    axes[1].plot(history['train_acc'], label='训练准确率', c='blue')
    axes[1].plot(history['val_acc'], label='验证准确率', c='orange', linestyle='--')
    axes[1].set_xlabel('Epoch'), axes[1].set_ylabel('Accuracy (%)'), axes[1].grid(True), axes[1].legend()
    
    plt.tight_layout(), plt.show()


以MNIST数据集为例，可视化训练一个中等隐私的CNN模型：

In [ ]:
# 设置差分隐私参数
dp_params = {
    'epsilon': 1.0,       # 隐私预算
    'delta': 1e-5,        # 高斯机制必需
    'sensitivity': 1.0,   # 梯度敏感度
    'mechanism': 'gaussian'  # 噪声类型
}

# 训练模型
model, history = train_dp_model(train_loader, val_loader, dp_params, epochs=8)

# 可视化训练过程
plot_training_history(history)

可以发现，使用差分隐私进行训练后，损失反而会随着每个Epoch的训练而增大，这是差分隐私训练导致的正常现象。

梯度裁剪、噪声都会对模型性能产生一定影响。所以在实际进行差分隐私训练时，需要做好性能和隐私保护的平衡。

接下来，我们对不同隐私配置进行差分隐私训练，观察模型的表现：

In [ ]:
# 对比不同隐私级别的实验
def compare_privacy_effect():
    """测试不同epsilon值对MNIST分类性能的影响"""
    
    # 加载数据
    train_loader, val_loader, test_loader = load_mnist_data(batch_size=128)
    
    # 隐私配置
    privacy_settings = [
        {'name': '无隐私', 'epsilon': float('inf'), 'delta': 0, 'mechanism': 'gaussian'},
        {'name': '弱隐私', 'epsilon': 10.0, 'delta': 1e-5, 'mechanism': 'gaussian'},
        {'name': '中隐私', 'epsilon': 1.0, 'delta': 1e-5, 'mechanism': 'gaussian'},
        {'name': '强隐私', 'epsilon': 0.1, 'delta': 1e-5, 'mechanism': 'gaussian'},
    ]
    
    # 运行实验
    results = []
    for cfg in privacy_settings:
        print(f"\n=== 训练 {cfg['name']} (ε={cfg['epsilon']}) ===")
        if cfg['epsilon'] == float('inf'):
            # 无隐私情况：直接训练
            model = build_cnn_model()
            optimizer = optim.Adam(model.parameters(), lr=0.001)
            for _ in range(10):
                run_training_epoch(model, train_loader, nn.CrossEntropyLoss(), optimizer,
                                  {'epsilon':1.0, 'delta':1e-5, 'sensitivity':1.0, 'mechanism':'gaussian'},
                                  max_grad=1.0)  # 不添加噪声
            _, test_acc = evaluate_model(model, test_loader, nn.CrossEntropyLoss())
            results.append({'name': cfg['name'], 'acc': test_acc, 'epsilon': cfg['epsilon']})
        else:
            # 差分隐私训练
            dp_cfg = {
                'epsilon': cfg['epsilon'],
                'delta': cfg['delta'],
                'sensitivity': 1.0,
                'mechanism': cfg['mechanism']
            }
            model, _ = train_dp_model(train_loader, val_loader, dp_cfg, lr=0.001, epochs=8)
            _, test_acc = evaluate_model(model, test_loader, nn.CrossEntropyLoss())
            results.append({'name': cfg['name'], 'acc': test_acc, 'epsilon': cfg['epsilon']})
    
    # 打印结果
    print("\n MNIST隐私保护对比：")
    for res in results:
        eps = "∞" if res['epsilon'] == float('inf') else f"{res['epsilon']:.1f}"
        print(f"{res['name']}: 测试准确率 {res['acc']:.2f}% (ε={eps})")

In [ ]:
# 对比不同隐私级别
compare_privacy_effect()